In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
import math

from tqdm import tqdm
from unidecode import unidecode 

from data_management.megadb.schema import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import *
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

## Uploading the eMammal sets

In [23]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_kays/emammal_kays_20190409.json') as f:
    kays_cct = json.load(f)

In [24]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_kays/emammal_kays_bboxes_20190409.json') as f:
    kays_boxes_cct = json.load(f)

In [25]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_long/emammal_long_20190409.json') as f:
    long_cct = json.load(f)

In [26]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_long/emammal_long_bboxes_20190409.json') as f:
    long_boxes_cct = json.load(f)

In [27]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_mcshea/emammal_mcshea_20190409.json') as f:
    mcshea_cct = json.load(f)

In [28]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_mcshea/emammal_mcshea_bboxes_20190409.json') as f:
    mcshea_boxes_cct = json.load(f)

In [11]:
sample(kays_cct['images'], 1)

[{'seq_num_frames': 3,
  'seq_id': 'd19135s194',
  'location': 'NCSU_GW09',
  'frame_num': 2,
  'width': 2592,
  'file_name': 'p101d19135/d19135s194i2.JPG',
  'id': 'datasetemammalkays.projectp101.deploymentd19135.seqd19135s194.imgd19135s194i2',
  'height': 1944,
  'datetime': '2016-03-05 09:47:51'}]

In [12]:
sample(long_cct['images'], 1)

[{'seq_num_frames': 3,
  'seq_id': 'd34787s464',
  'location': '2786-5',
  'frame_num': 2,
  'width': 2048,
  'file_name': 'p158d34787/d34787s464i2.JPG',
  'id': 'datasetemammallong.projectp158.deploymentd34787.seqd34787s464.imgd34787s464i2',
  'height': 1536,
  'datetime': '2017-01-29 09:02:23'}]

In [13]:
sample(mcshea_cct['images'], 1)

[{'seq_num_frames': 40,
  'seq_id': 'd21464s23',
  'location': 'VWL Smith Property Ru 1_16',
  'frame_num': 24,
  'width': 2048,
  'file_name': '3191d21464/d21464s23i24.JPG',
  'id': 'datasetemammalmcshea.project3191.deploymentd21464.seqd21464s23.imgd21464s23i24',
  'height': 1536,
  'datetime': '2016-09-21 17:39:01'}]

In [29]:
kays_folders = set([i['file_name'].split('/')[0] for i in kays_cct['images']])

In [30]:
long_folders = set([i['file_name'].split('/')[0] for i in long_cct['images']])

In [31]:
mcshea_folders = set([i['file_name'].split('/')[0] for i in mcshea_cct['images']])

In [32]:
len(kays_folders)

2130

In [33]:
len(long_folders)

126

In [34]:
len(mcshea_folders)

1010

In [35]:
kays_folders.intersection(long_folders)
kays_folders.intersection(mcshea_folders)

set()

set()

In [36]:
mcshea_folders.intersection(long_folders)

set()

In [37]:
emammal_folders = {
    'wpz_emammal_2018': list(long_folders),
    'mcshea_emammal': list(mcshea_folders),
    'kays_emammal': list(kays_folders)
}

In [39]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/databases_201904/emammal_folders.json', 'w') as f:
    json.dump(emammal_folders, f, indent=4)

## wpz_emammal_2018

This is the Robert Long set.

In [22]:
wpz_embedded = make_cct_embedded(image_db=long_cct, bbox_db=long_boxes_cct)

Loading image DB...
Number of items from the image DB: 98486
Number of images with more than 1 species: 17010 (17.27% of image DB)
Loading bbox DB...
Number of images added from bbox DB entries:  0
Number of images amended:  0
Number of items in total:  98486
Number of images with more than one bounding box: 547 (0.5554088906037407% of all entries)


## mcshea_emammal

Problem with `seq_id` - this is only unique within a deployment. Fixed below.

`'id': 'datasetemammalmcshea.projectp195.deploymentL-HY11-070.seq1.imgL-HY11-070-003'`

In [6]:
mcshea_embedded = make_cct_embedded(image_db=mcshea_cct, bbox_db=mcshea_boxes_cct)

Loading image DB...
Number of items from the image DB: 504433
Number of images with more than 1 species: 4794 (0.95% of image DB)
Loading bbox DB...
Number of images added from bbox DB entries:  0
Number of images amended:  0
Number of items in total:  504433
Number of images with more than one bounding box: 343 (0.06799713737998901% of all entries)


In [7]:
sample(mcshea_embedded, 1)

[{'seq_num_frames': 15,
  'seq_id': 'd31825s27',
  'location': 'VWL Shelby Ru 7_17',
  'frame_num': 2,
  'width': 2048,
  'file_name': '3191d31825/d31825s27i2.JPG',
  'id': 'datasetemammalmcshea.project3191.deploymentd31825.seqd31825s27.imgd31825s27i2',
  'height': 1536,
  'datetime': '2017-08-19 17:05:51',
  'annotations': {'species': ['domestic cow']}}]

In [8]:
for e in mcshea_embedded:
    parts = e['id'].split('.')
    correct_seq_id = '.'.join(parts[1:4])
    e['seq_id'] = correct_seq_id

In [9]:
sample(mcshea_embedded, 1)

[{'seq_num_frames': 60,
  'seq_id': 'project3191.deploymentd19850.seqd19850s60',
  'location': 'VWL Stonebridge Ru 1_16',
  'frame_num': 27,
  'width': 2048,
  'file_name': '3191d19850/d19850s60i27.JPG',
  'id': 'datasetemammalmcshea.project3191.deploymentd19850.seqd19850s60.imgd19850s60i27',
  'height': 1536,
  'datetime': '2016-06-29 11:54:10',
  'annotations': {'species': ['empty']}}]

In [15]:
# sequences with frame_num that are not unique

problem_sequences = []

for seq in sequences:
    if 'images' not in seq:
        continue
        
    # if there are more than one image item, each needs a frame_num
    if len(seq['images']) > 1:
        frame_num_set = []
        for i in seq['images']:
            if 'frame_num' not in i:
                assert False, 'sequence {} has more than one image but not all images have frame_num'.format(seq['seq_id'])

            frame_num_set.append(i['frame_num'])
            
        if len(set(frame_num_set)) != len(seq['images']):
            problem_sequences.append(seq)

In [16]:
len(problem_sequences)

1

In [27]:
# seems like the deployment info in the id field is wrong (according to file name) - no bbox - drop this.
mcshea_sequences = []

for seq in sequences:
    if seq['seq_id'] != 'project3191.deploymentd17359.seqd17359s3':
        mcshea_sequences.append(seq)

In [29]:
for seq in mcshea_sequences:
    if 'class' in seq:
        class_norm = []
        for c in seq['class']:
            class_norm.append(unidecode(c))
        seq['class'] = class_norm

In [30]:
sequences_schema_check.sequences_schema_check(mcshea_sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [31]:
write_json(out_path, mcshea_sequences)

## kays_emammal

In [9]:
kays_embedded = make_cct_embedded(image_db=kays_cct, bbox_db=kays_boxes_cct)

Loading image DB...
Number of items from the image DB: 1155991
Number of images with more than 1 species: 36329 (3.14% of image DB)
Loading bbox DB...
Number of images added from bbox DB entries:  0
Number of images amended:  9477
Number of items in total:  1155991
Number of images with more than one bounding box: 1529 (0.13226746575016587% of all entries)


In [10]:
len(kays_embedded)
sample(kays_embedded, 1)

1155991

[{'seq_num_frames': 45,
  'seq_id': 'd26309s34',
  'location': 'Harrison_D_Greene4',
  'frame_num': 19,
  'width': 1920,
  'file_name': 'p168d26309/d26309s34i19.JPG',
  'id': 'datasetemammalkays.projectp168.deploymentd26309.seqd26309s34.imgd26309s34i19',
  'height': 1080,
  'datetime': '2017-06-12 03:45:44',
  'annotations': {'species': ['white-tailed deer']}}]

In [11]:
for e in kays_embedded:
    parts = e['id'].split('.')
    correct_seq_id = '.'.join(parts[1:4])
    e['seq_id'] = correct_seq_id

In [12]:
sample(kays_embedded, 1)

[{'seq_num_frames': 3,
  'seq_id': 'projectp168.deploymentd21818.seqd21818s201',
  'location': 'Caveny_J_Avery1',
  'frame_num': 1,
  'width': 1920,
  'file_name': 'p168d21818/d21818s201i1.JPG',
  'id': 'datasetemammalkays.projectp168.deploymentd21818.seqd21818s201.imgd21818s201i1',
  'height': 1440,
  'datetime': '2016-12-21 07:48:43',
  'annotations': {'species': ['white-tailed deer']}}]

In [17]:
# some frame_num were not unique

problem_sequences = []

for seq in sequences:
    if 'images' not in seq:
        continue
        
    # if there are more than one image item, each needs a frame_num
    if len(seq['images']) > 1:
        frame_num_set = []
        for i in seq['images']:
            if 'frame_num' not in i:
                assert False, 'sequence {} has more than one image but not all images have frame_num'.format(seq['seq_id'])

            frame_num_set.append(i['frame_num'])
            
        if len(set(frame_num_set)) != len(seq['images']):
            problem_sequences.append(seq)

In [20]:
# only one had issues, no bbox, dropping it

kays_sequences = []
for e in sequences:
    if e['seq_id'] != 'projectp168.deploymentd22148.seqd22148s113':
        kays_sequences.append(e)

In [21]:
sequences_schema_check.sequences_schema_check(kays_sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [22]:
write_json(out_path, kays_sequences)

## Make MegaDB

In [13]:
embedded = kays_embedded
dataset_name = 'kays_emammal'
out_path = '/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11/{}.json'.format(dataset_name)

In [15]:
sequences = process_sequences(embedded, dataset_name)

The dataset_name is set to kays_emammal. Please make sure this is correct!
Making a deep copy of docs...


  6%|▋         | 72887/1155991 [00:00<00:02, 364382.35it/s]

Putting 1155991 images into sequences...


100%|██████████| 1155991/1155991 [00:03<00:00, 380572.31it/s]


Number of sequences: 106755
Checking the location field...
Checking which fields in a CCT image entry are sequence-level...

all_img_properties
{'id', 'datetime', 'class', 'bbox', 'frame_num', 'location', 'file'}

img_level_properties
{'id', 'datetime', 'bbox', 'frame_num', 'file'}

image-level properties that really should be sequence-level
{'class', 'location'}

Finished processing sequences.
Example sequence items:

{'seq_id': 'projectp168.deploymentd22883.seqd22883s1', 'dataset': 'kays_emammal', 'images': [{'frame_num': 1, 'id': 'datasetemammalkays.projectp168.deploymentd22883.seqd22883s1.imgd22883s1i1', 'datetime': '2017-03-26 08:09:37', 'file': 'p168d22883/d22883s1i1.JPG'}, {'frame_num': 2, 'id': 'datasetemammalkays.projectp168.deploymentd22883.seqd22883s1.imgd22883s1i2', 'datetime': '2017-03-26 08:09:38', 'file': 'p168d22883/d22883s1i2.JPG'}, {'frame_num': 3, 'id': 'datasetemammalkays.projectp168.deploymentd22883.seqd22883s1.imgd22883s1i3', 'datetime': '2017-03-26 08:09:39', 'fi

In [ ]:
sequences_schema_check.sequences_schema_check(sequences)

In [34]:
write_json(out_path, sequences)